# Financial ML System - Neural Network Training

This notebook trains the neural network for return prediction.

## 1. Setup

In [ ]:
import sys
import warnings
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, callbacks
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error

warnings.filterwarnings('ignore')
tf.get_logger().setLevel('ERROR')

PROJECT_ROOT = Path('/content/financial-ml-system')
sys.path.insert(0, str(PROJECT_ROOT))

from src.utils.constants import DATA_DIR, MODELS_DIR
from src.utils.config_loader import config

print(f"TensorFlow version: {tf.__version__}")
print(f"GPU available: {len(tf.config.list_physical_devices('GPU')) > 0}")

## 2. Load Features

In [ ]:
ticker = config.get('data.default_ticker', 'SPY')
features_file = DATA_DIR / 'processed' / f"{ticker}_features.csv"

data = pd.read_csv(features_file, index_col=0, parse_dates=True)
print(f"Loaded {len(data)} rows")
data.head()

## 3. Prepare Data for Neural Network

In [ ]:
# Select features
feature_cols = [
    'Returns', 'SMA_Ratio_5', 'SMA_Ratio_20', 'SMA_Ratio_50',
    'RSI', 'MACD', 'MACD_Hist',
    'BB_Width', 'BB_Position',
    'ATR_Pct', 'Volatility',
    'Volume_Ratio', 'ROC', 'Momentum'
]

missing = [col for col in feature_cols if col not in data.columns]
if missing:
    print(f"Warning: Missing features: {missing}")
    feature_cols = [col for col in feature_cols if col in data.columns]

# Create target: next period return
data['Target'] = data['Returns'].shift(-1)
data = data.dropna(subset=['Target'])

X = data[feature_cols].values
y = data['Target'].values

print(f"Features shape: {X.shape}")
print(f"Target shape: {y.shape}")
print(f"Target mean: {y.mean():.6f}")
print(f"Target std: {y.std():.6f}")

## 4. Train-Test Split

In [ ]:
train_size = config.get('data.train_test_split', 0.8)
split_idx = int(len(X) * train_size)

X_train, X_test = X[:split_idx], X[split_idx:]
y_train, y_test = y[:split_idx], y[split_idx:]

print(f"Training set: {X_train.shape[0]} samples")
print(f"Test set: {X_test.shape[0]} samples")

## 5. Feature Scaling

In [ ]:
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("Features scaled to [0, 1]")
print(f"Training set min: {X_train_scaled.min():.4f}")
print(f"Training set max: {X_train_scaled.max():.4f}")

## 6. Build Neural Network

In [ ]:
def build_model(input_dim, layers_config, dropout_rates, learning_rate):
    """Build neural network model."""
    model = models.Sequential()
    
    # Input layer
    model.add(layers.Input(shape=(input_dim,)))
    
    # Hidden layers
    for neurons, dropout in zip(layers_config, dropout_rates):
        model.add(layers.Dense(neurons, activation='relu'))
        if dropout > 0:
            model.add(layers.Dropout(dropout))
    
    # Output layer
    model.add(layers.Dense(1, activation='linear'))
    
    # Compile
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        loss='mse',
        metrics=['mae']
    )
    
    return model

# Get configuration
layers_config = config.get('models.nn.layers', [64, 32, 16])
dropout_rates = config.get('models.nn.dropout', [0.3, 0.2, 0.0])
learning_rate = config.get('models.nn.learning_rate', 0.001)

# Build model
model = build_model(
    input_dim=X_train_scaled.shape[1],
    layers_config=layers_config,
    dropout_rates=dropout_rates,
    learning_rate=learning_rate
)

print(model.summary())

## 7. Train Model

In [ ]:
# Callbacks
early_stopping = callbacks.EarlyStopping(
    monitor='val_loss',
    patience=config.get('models.nn.early_stopping_patience', 10),
    restore_best_weights=True,
    verbose=1
)

reduce_lr = callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=5,
    min_lr=1e-7,
    verbose=1
)

# Train
history = model.fit(
    X_train_scaled, y_train,
    validation_split=0.2,
    epochs=config.get('models.nn.epochs', 100),
    batch_size=config.get('models.nn.batch_size', 32),
    callbacks=[early_stopping, reduce_lr],
    verbose=1
)

print("\nTraining complete")

## 8. Training History

In [ ]:
# Plot training history
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Loss
ax1.plot(history.history['loss'], label='Training Loss')
ax1.plot(history.history['val_loss'], label='Validation Loss')
ax1.set_title('Model Loss', fontsize=14, fontweight='bold')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss (MSE)')
ax1.legend()
ax1.grid(True, alpha=0.3)

# MAE
ax2.plot(history.history['mae'], label='Training MAE')
ax2.plot(history.history['val_mae'], label='Validation MAE')
ax2.set_title('Model MAE', fontsize=14, fontweight='bold')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('MAE')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(PROJECT_ROOT / 'results' / 'nn_training_history.png', dpi=300, bbox_inches='tight')
plt.show()

## 9. Evaluation

In [ ]:
# Predictions
train_pred = model.predict(X_train_scaled, verbose=0).flatten()
test_pred = model.predict(X_test_scaled, verbose=0).flatten()

# Metrics
train_mae = mean_absolute_error(y_train, train_pred)
train_mse = mean_squared_error(y_train, train_pred)
train_rmse = np.sqrt(train_mse)

test_mae = mean_absolute_error(y_test, test_pred)
test_mse = mean_squared_error(y_test, test_pred)
test_rmse = np.sqrt(test_mse)

print("Performance Metrics")
print("=" * 50)
print(f"Training MAE:  {train_mae:.6f}")
print(f"Training RMSE: {train_rmse:.6f}")
print(f"Test MAE:      {test_mae:.6f}")
print(f"Test RMSE:     {test_rmse:.6f}")

In [ ]:
# Directional accuracy
train_direction = np.sign(train_pred) == np.sign(y_train)
test_direction = np.sign(test_pred) == np.sign(y_test)

print("\nDirectional Accuracy")
print("=" * 50)
print(f"Training: {train_direction.mean():.4f}")
print(f"Test:     {test_direction.mean():.4f}")

In [ ]:
# Prediction vs Actual
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Scatter plot
ax1.scatter(y_test, test_pred, alpha=0.5, s=10)
ax1.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 
         'r--', lw=2, label='Perfect Prediction')
ax1.set_title('Predicted vs Actual Returns', fontsize=14, fontweight='bold')
ax1.set_xlabel('Actual Returns')
ax1.set_ylabel('Predicted Returns')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Error distribution
errors = test_pred - y_test
ax2.hist(errors, bins=50, edgecolor='black', alpha=0.7)
ax2.axvline(x=0, color='r', linestyle='--', linewidth=2)
ax2.set_title('Prediction Error Distribution', fontsize=14, fontweight='bold')
ax2.set_xlabel('Prediction Error')
ax2.set_ylabel('Frequency')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(PROJECT_ROOT / 'results' / 'nn_predictions.png', dpi=300, bbox_inches='tight')
plt.show()

## 10. Save Model

In [ ]:
import joblib
import json

# Save neural network
model_dir = MODELS_DIR / 'nn'
model_dir.mkdir(parents=True, exist_ok=True)

model_path = model_dir / 'nn_predictor.h5'
scaler_path = model_dir / 'scaler.pkl'

model.save(model_path)
joblib.dump(scaler, scaler_path)

# Save model info
model_info = {
    'feature_names': feature_cols,
    'train_mae': float(train_mae),
    'test_mae': float(test_mae),
    'train_rmse': float(train_rmse),
    'test_rmse': float(test_rmse),
    'train_direction_acc': float(train_direction.mean()),
    'test_direction_acc': float(test_direction.mean()),
    'architecture': {
        'layers': layers_config,
        'dropout': dropout_rates,
        'learning_rate': learning_rate
    }
}

with open(model_dir / 'model_info.json', 'w') as f:
    json.dump(model_info, f, indent=2)

print(f"Model saved to: {model_path}")
print(f"Scaler saved to: {scaler_path}")
print(f"Model info saved to: {model_dir / 'model_info.json'}")

## Neural Network Training Complete

Next: Open `05_signal_generation.ipynb`